In [ ]:
!pip install textattack
!pip install tensorflow_text

In [ ]:
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper

# https://huggingface.co/textattack
model = transformers.AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/outputs/fewshotdeberta")
tokenizer = transformers.AutoTokenizer.from_pretrained("/content/drive/MyDrive/outputs/fewshotdeberta")
# We wrap the model so it can be used by textattack
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

In [ ]:
def loadData(source_path, target_path):
    custom_dataset = []
    with open(source_path) as F:
        with open(target_path) as G:
            f=F.read().split('\n')
            g=G.read().split('\n')
            for i in range(len(f)):
                if ('2' in g[i]):
                    custom_dataset+=[(f[i], 2)]
    return custom_dataset

custom_dataset = loadData("/content/drive/MyDrive/outputs/test_sentence.source", "/content/drive/MyDrive/outputs/test_sentence.target")

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='/content/drive/MyDrive/outputs/fewshotdeberta', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'bos_token': AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=True)})

In [ ]:
# dataset: An iterable of (text, ground_truth_output) pairs.
#0 means the review is negative
#1 means the review is positive
custom_dataset = loadData("/content/drive/MyDrive/outputs/test_sentence.source", "/content/drive/MyDrive/outputs/test_sentence.target")

In [ ]:
custom_dataset[0][0]

'the text has the logical fallacy of cum hoc ergo propter hoc </s> since lucy sutton became vice president of the parent-teacher association, student performance has declined and teacher morale is down. we on the school board believe that sutton bears sole responsibility for the downtrend.'

In [ ]:
len(custom_dataset)

305

In [ ]:
from textattack import Attack
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.goal_functions import UntargetedClassification
from textattack.transformations import WordSwapEmbedding
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification


textattack: Unknown if model of class <class 'transformers.models.deberta.modeling_deberta.DebertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
import textattack

In [ ]:
for example, label in custom_dataset:
    result = attack_hotflip.attack(example, label)
    print(result.__str__(color_method='ansi'))
    print("###################################")

In [ ]:
attack_InputReduction = textattack.attack_recipes.input_reduction_feng_2018.InputReductionFeng2018.build(model_wrapper)

textattack: Unknown if model of class <class 'transformers.models.deberta.modeling_deberta.DebertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.input_reduction.InputReduction'>.


In [ ]:
import torch

In [ ]:
fail_case = []
for i in custom_dataset[:]:
    ans=model(torch.tensor([tokenizer(i[0]).input_ids]).cuda()).logits
    if (ans[0][2]!=torch.max(ans)):
        fail_case.append((i[0],0))
        fail_case.append((i[0],1))

In [ ]:
len(fail_case)

142

In [ ]:
len(custom_dataset)

305

In [ ]:
def calcStr(str1):
    str1=str1.replace("\x1b[91m","<del>")
    str1=str1.replace("\x1b[0m","</del>")
    str1=str1.replace("</del> <del>"," ")
    return generate_InputReductionText(str1)
def generate_InputReductionText(str1):
    str1=str1.replace("%", "\%")
    str1=str1.replace("-->", "to")
    str1=str1.replace("<del>s</del>", "s")
    str1=str1.replace("<<del>/s</del>>", "</s>")
    str1=str1.replace("</s>", "[SEP]")
    str1=str1.replace("</>", "[SEP]")
    str1=str1.replace("<del>", "\sout{")
    str1=str1.replace("</del>", "}")
    return str1


In [ ]:
str1

'\\sout{2 (100%)} --> \\sout{2 (78%)}\n\n\\sout{the} text \\sout{has the logical fallacy} of cum \\sout{hoc ergo propter} hoc </s> \\sout{since} lucy \\sout{sutton} became vice president \\sout{of the} parent-teacher \\sout{association}, student \\sout{performance} has declined and \\sout{teacher morale is} down. \\sout{we on the} school \\sout{board believe that sutton bears sole responsibility} for the downtrend.\n\ntext of cum hoc </s> lucy became vice president parent-teacher, student has declined and down. school for the downtrend.'

In [ ]:
li=[]

In [ ]:
ans=0
for example, label in fail_case:
    result = attack_InputReduction.attack(example, label)
    li.append(calcStr(result.__str__(color_method='ansi')))
    ans+=1
    if (ans%10==0):
        print(ans)

10
20
30
40
50
60
70
80
90
100
110
120
130
140


In [ ]:
def generate_InputReductionText(str1):
    str1=str1.replace("%", "\%")
    str1=str1.replace("-->", "to")
    str1=str1.replace("<del>s</del>", "s")
    str1=str1.replace("<<del>/s</del>>", "</s>")
    str1=str1.replace("</s>", "[SEP]")
    str1=str1.replace("</>", "[SEP]")
    str1=str1.replace("<del>", "\sout{")
    str1=str1.replace("</del>", "}")
    return str1


In [ ]:
text, lii = [], []
for i in range(len(fail_case)):
    if (not "SKIPPED" in li[i] and not "FAIL" in li[i]):
        lii.append(li[i])
        text.append(fail_case[i])

In [ ]:
str1=lii[20]

In [ ]:
def generate_qwq(str1):
    return ("{\setlength{\\fboxsep}{0pt}\colorbox{white!0}{\parbox{0.9\\textwidth}{\n"+str1.replace("\sout{","\colorbox{gray!70}{\strut ")+"\n}}}\n\n")


In [ ]:
str1 = ""
for i in range(len(lii)):
    str1+=text[i][0]+"\n\n"
    str1+=generate_qwq(lii[i])
str1

'the text has the logical fallacy of circular </s> you would think that taxes should be lowered because you are a republican [and therefore your argument about taxes should be rejected].\n\n{\\setlength{\\fboxsep}{0pt}\\colorbox{white!0}{\\parbox{0.9\\textwidth}{\n\\colorbox{gray!70}{\\strut 0 (99\\%)} to \\colorbox{gray!70}{\\strut 0 (95\\%)}\n\n\\colorbox{gray!70}{\\strut the} text \\colorbox{gray!70}{\\strut has the logical} fallacy of circular [SEP] you \\colorbox{gray!70}{\\strut would think that taxes should} be lowered \\colorbox{gray!70}{\\strut because you are a republican} [\\colorbox{gray!70}{\\strut and therefore} your \\colorbox{gray!70}{\\strut argument} about \\colorbox{gray!70}{\\strut taxes should be} rejected].\n\ntext fallacy of circular [SEP] you be lowered [ your about rejected].\n}}}\n\nthe text has the logical fallacy of base rate </s> faith healing "works," but not all the time, especially when one’s faith is not strong enough (as generally indicated by the size

In [ ]:
import pandas as pd
dict_qwq={"Text":text, 
                      "InputReduction_Answer":lii, 
                    }
datas = pd.DataFrame(dict_qwq)
datas.to_csv('inputReduction_eletra.csv')